In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
nbpath     = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN'
voxel_path = '/home/mgolub4/DLproj/MLTO_2024/0_data/voxel_arrays_npy_by_partno'

# dbpath = os.path.join(nbpath, 'dyn_data', 'dynamic_static_database_scaled_DEC23_constit_eqn_fit_params.csv')
dbpath = os.path.join(nbpath, 'dyn_data', 'dyn_stat_database_PINN_ready.csv') # dynamic_static_database_scaled_APR24_constit_eqn_fit_params


dyndb = pd.read_csv(dbpath)

stress_ser_path = os.path.join(nbpath, 'dyn_data/stress_series_data')

# csvs = [p for p in sorted(os.listdir(stress_ser_path)) if p.endswith('untrunc.csv')]
csvs = [p for p in sorted(os.listdir(stress_ser_path)) if p.endswith('_proct_gaus_btrlp_fftlp.csv')]


In [3]:
# Ensuring that I only pull the csvnames of the actual simulation data that I use in the dynamic database (100 of 102)

truncsvs = []
for csv in csvs:
    truncsvs.append(csv.rsplit('_',4)[0])

dyndbcsvs = [name for name in truncsvs if name in dyndb['dyn_file_name_original'].values]

len(dyndbcsvs)

100

In [4]:
newcols = ['plateau_stress_g_from_scaled', 'energy_absorbed_g_from_scaled', 'sig_pl_offsetmask', 'sig_pl_offsetmask_scaled', 'W_pred_offsetmask', 'W_pred_offsetmask_scaled']

for col in newcols:
    dyndb[col] = pd.Series(np.zeros(len(dyndb)))

In [5]:
coeffcols =[ 'A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt',]
offset = -0.01

In [ ]:
# dyndb[dyndb['dyn_file_name_original'] == 'VF_35_64_VF35p92'][coeffcols]

In [ ]:
# dyndb[dyndb['dyn_file_name_original'] == name][coeffcols]['A_opt'].values[0]

In [ ]:
# cutting off negative values for strain once offset is applied

# mask = strain >= offset*-1

# strainoff = strain + offset

# strainoff = np.where(mask, strainoff, 0)

In [7]:
derived_data_params_scaled = ['plateau_stress_g_from_scaled', 'energy_absorbed_g_from_scaled',]

for name in dyndbcsvs:
    series_csvname = name + '_proct_gaus_btrlp_fftlp.csv'
    new_filename = name + '_proct_w_constit_eqn_and_scaled_series.csv'
    colname = name

    dyndb_index = dyndb[dyndb['dyn_file_name_original'] == name].index[0]
    
    series_csvpath = os.path.join(stress_ser_path, series_csvname)
    newseries_csvpath = os.path.join(stress_ser_path, new_filename)

    # print(series_csvpath)

    seriesdf = pd.read_csv(series_csvpath)

    # 1 (ha, ha, ha...) Min-Max scale the simulation data, add it back to the csv
    stress_data = seriesdf['stress_bottom_gsreg']
    strain = seriesdf['Strain']

    mask = strain >= offset*-1
    strainoff = strain + offset
    strainoff = np.where(mask, strainoff, 0)

    data_min = stress_data.min()
    data_max = stress_data.max()

    stress_data_scaled = (stress_data - data_min) / (data_max - data_min)

    seriesdf['stress_bottom_gsreg_scaled'] = stress_data_scaled
    
    # # 1a save as the original, the equivalent of just adding a column --> LATER
    # seriesdf.to_csv(series_csvpath)

    #  2 (ha, ha, ha...) calculate sig_pl and W of scaled data
    sig_pl_data_scaled = np.mean(stress_data_scaled[200:400])
    W_data_scaled= np.trapz(stress_data_scaled, strain)

    # 2a enter into dyndb

    for col, val in zip(derived_data_params_scaled, [sig_pl_data_scaled, W_data_scaled]):
        # dyndb[dyndb['dyn_file_name_original'] == name][col] = val
        dyndb.loc[dyndb_index, col] = val


    # 3 - constitutive equation calculations and derived parameters
    # 3a calc constitutive equation stress series and derived parameters

    coeffs = dyndb[dyndb['dyn_file_name_original'] == name][coeffcols]
    
    A_opt = coeffs['A_opt'].values[0]
    B_opt = coeffs['B_opt'].values[0]
    C_opt = coeffs['C_opt'].values[0]
    m_opt = coeffs['m_opt'].values[0]
    n_opt = coeffs['n_opt'].values[0]

    constit_eqn_stress = A_opt * strainoff**m_opt + B_opt * (strainoff / (C_opt - strainoff))**n_opt
    
    # ... add to time series dataframe
    seriesdf['stress_bottom_constitutive_equation'] = constit_eqn_stress


    # 3b scale series
    # stress_max = constit_eqn_stress.max()
    # stress_min = constit_eqn_stress.min()

    # constit_eqn_stress_scaled = (constit_eqn_stress - stress_min) / (stress_max - stress_min)
    constit_eqn_stress_scaled = (constit_eqn_stress - data_min) / (data_max - data_min) # "data_min" and "data_max" are from the source data --> since the constitutive equation is to have the same scale as the source data, they should be scaled from the source data's max and min
    # ... add to dataframe
    seriesdf['stress_bottom_constitutive_equation_scaled'] = constit_eqn_stress_scaled

    # ... and save
    seriesdf.to_csv(newseries_csvpath)

    # 3c calculated sig_pl and W from constitutive equation --> un/scaled
    #  'sig_pl_offsetmask', 'sig_pl_offsetmask_scaled', 'W_pred_offsetmask', 'W_pred_offsetmask_scaled']
    sig_pl_eqn = np.mean(constit_eqn_stress[200:400])
    sig_pl_eqn_scaled = np.mean(constit_eqn_stress_scaled[200:400])
    W_eqn = np.trapz(constit_eqn_stress, strainoff)
    W_eqn_scaled = np.trapz(constit_eqn_stress_scaled, strainoff)

    # ... add to dyndb
    for col, val in zip(['sig_pl_offsetmask', 'sig_pl_offsetmask_scaled', 'W_pred_offsetmask', 'W_pred_offsetmask_scaled'],
                        [sig_pl_eqn, sig_pl_eqn_scaled, W_eqn, W_eqn_scaled]):
        # dyndb[dyndb['dyn_file_name_original'] == name][col] = val
        dyndb.loc[dyndb_index, col] = val

    
 

In [8]:
dyndb.to_csv(dbpath)

## Scaling by absolute min and max (min and max across all stress series), as I did below,  doesn't work

In [ ]:
stressdf.min().min()

In [ ]:
absmax = stressdf.max().max()
print(absmax)

In [ ]:
absmin = stressdf.min().min()
print(absmin)

In [ ]:
for col in stressdf.columns:
    
    col_scaled = (stressdf[col] - absmin) / (absmax - absmin)

    stressdf[f'{col}_abscaled'] = col_scaled

In [ ]:
stressdf